# 题目描述：

泰坦尼克海难是著名的十大灾难之一，究竟多少人遇难，各方统计的结果不一。

现在我们可以得到部分的数据

其中数据集格式为 csv，一共有两个文件：

train.csv 是训练数据集，包含特征信息和存活与否的标签；

test.csv: 测试数据集，只包含特征信息。

集合中具体数据说明：

字段名|描述|
--|:--:|
PassengerId|乘客编号|
Survived|存活与否|
Pclass|船票登记|
Name|姓名|
Sex|性别|
Age|年龄|
SibSp|亲戚数量(兄弟，配偶)|
Parch|亲戚数量(子女)|
Ticket|船票号码|
Fare|票价|
Cabin|船仓|
Embarked|登陆港口|

现在我们需要用决策树分类对训练集进行训练，针对测试集中的乘客进行生存预测，并告知分类器的准确率。

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer

In [2]:
# 读取数据
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
train_data.info()

FileNotFoundError: [Errno 2] File ./train.csv does not exist: './train.csv'

In [ ]:
train_data.head()

In [ ]:
#panda 提供的一些功能
train_data.describe() 

In [ ]:
# 查看字符串类型（非数字）的整体情况
train_data.describe(include=['O'])

In [ ]:
train_data.tail()

## 数据准备

之前我们通过调用panda 中的方法，对数据内容有一个大体的了解。 

也很容易发现 Age、Fare 和 Cabin 这三个字段的数据有所缺失。

其中 Age 为年龄字段，是数值型，我们可以通过平均值进行补齐；
Fare 为船票价格，是数值型，我们也可以通过其他人购买船票的平均值进行补齐。

In [ ]:
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
# 使用票价的均值填充票价中的nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

In [ ]:
train_data['Embarked'].value_counts()

我们发现一共就 3 个登陆港口，其中 S 港口人数最多，占到了 72%，因此我们将其余缺失的 Embarked 数值均设置为 S：

In [ ]:
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)

## 特征选择特征

选择是分类器的关键。特征选择不同，得到的分类器也不同。
显然，PassengerId,Name是不会影响乘客能否生存，可以不选择；
可以放弃；Cabin 字段缺失值太多，可以放弃；Ticket 字段为船票号码，杂乱无章且无规律，可以放弃。

### 特征值处理
其余的字段包括：Pclass、Sex、Age、SibSp、Parch 和 Fare，这些属性分别表示了乘客的船票等级、性别、年龄、亲戚数量以及船票价格，可能会和乘客的生存预测分类有关系。
具体是什么关系，我们可以交给分类器来处理。因此我们先将 Pclass、Sex、Age 等这些其余的字段作特征，
放到特征向量 features 里。

In [ ]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

train_features

特征值里有一些是字符串，这样不方便后续的运算，需要转成数值类型，比如 Sex 字段，有 male 和 female 两种取值。我们可以把它变成 Sex=male 和 Sex=female 两个字段，数值用 0 或 1 来表示。同理 Embarked 有 S、C、Q 三种可能，我们也可以改成 Embarked=S、Embarked=C 和 Embarked=Q 三个字段，数值用 0 或 1 来表示。那该如何操作呢，我们可以使用 sklearn 特征选择中的 DictVectorizer 类，用它将可以处理符号化的对象，将符号转成数字 0/1 进行表示。具体方法如下：

In [ ]:
dvec=DictVectorizer(sparse=False)
object_dic=train_features.to_dict(orient='record') # panda里没一行转化成字典
train_features=dvec.fit_transform(object_dic)
train_features[1]# pandas 里的数据放入np.arrays 里

In [ ]:
dvec.feature_names_

## 构造ID3决策树

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# 构造ID3决策树
clf = DecisionTreeClassifier(criterion='entropy')
# 决策树训练
clf.fit(train_features, train_labels)

In [ ]:
test_features=dvec.transform(test_features.to_dict(orient='record'))
# 决策树预测
pred_labels = clf.predict(test_features)

In [ ]:
# 得到决策树准确率
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score准确率为 %.4lf' % acc_decision_tree)

In [ ]:

import numpy as np
from sklearn.model_selection import cross_val_score
# 使用K折交叉验证 统计决策树准确率
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))